#  APIs for Air Quality Data

---

### Part A

In this section, I will request and analyze air quality data from OpenWeather's API (https://openweathermap.org/api).

In [7]:
import pandas as pd
import json
import plotly.express as px
import requests

In [8]:
# The API key is stored separately on a config file for security reasons
with open("config.json") as f:
    config = json.load(f)
    my_key = config["my_key"]

Retrieve the API for weather data for Chicago.

In [9]:
city_name = 'Chicago'
country_code = 'US'
weather = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={city_name},{country_code}&appid={my_key}')
weather.json()

{'coord': {'lon': -87.65, 'lat': 41.85},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 292.77,
  'feels_like': 292.64,
  'temp_min': 292.77,
  'temp_max': 292.77,
  'pressure': 1022,
  'humidity': 71,
  'sea_level': 1022,
  'grnd_level': 1001},
 'visibility': 10000,
 'wind': {'speed': 6.8, 'deg': 41, 'gust': 9.2},
 'clouds': {'all': 52},
 'dt': 1753969967,
 'sys': {'country': 'US', 'sunrise': 1753958617, 'sunset': 1754010610},
 'timezone': -18000,
 'id': 4887398,
 'name': 'Chicago',
 'cod': 200}

Access the coordinates and weather data in the JSON and assign them to separate variables.

In [10]:
# Store the API data under 'coord' and 'main' into separate dictionaries
weather_coord = weather.json()['coord']
weather_main = weather.json()['main']
print(weather_coord)
print(weather_main)

{'lon': -87.65, 'lat': 41.85}
{'temp': 292.77, 'feels_like': 292.64, 'temp_min': 292.77, 'temp_max': 292.77, 'pressure': 1022, 'humidity': 71, 'sea_level': 1022, 'grnd_level': 1001}


Retrieve the air quality data for Chicago.

In [11]:
air_quality = requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution?lat={weather_coord['lat']}&lon={weather_coord['lon']}&appid={my_key}')
air_quality.json()

{'coord': {'lon': -87.65, 'lat': 41.85},
 'list': [{'main': {'aqi': 1},
   'components': {'co': 165.54,
    'no': 0.27,
    'no2': 6.4,
    'o3': 19.11,
    'so2': 0.65,
    'pm2_5': 7.87,
    'pm10': 7.91,
    'nh3': 0.27},
   'dt': 1753970209}]}

The goal is to create a large dataset of weather data for various cities. <br>

The first step is to create a large dictionary that the following data for Chicago:

* City name (Chicago, duh)
* Lat/long coordinates
* All data contained in `main` from the weather output
* The air quality index (AQI) (contained in the air pollution data)
* All data contained in `components` from the air pollution output

In [12]:
# Store the API data under 'aqi' and 'components' into separate variable and dictionary
air_quality_aqi = air_quality.json()['list'][0]['main']['aqi']
air_quality_components = air_quality.json()['list'][0]['components']

# Merge all dictionaries and variables into a single dictionary
weather_chicago = weather_coord | weather_main | air_quality_components
weather_chicago['city name'] = city_name
weather_chicago['aqi'] = air_quality_aqi
weather_chicago

{'lon': -87.65,
 'lat': 41.85,
 'temp': 292.77,
 'feels_like': 292.64,
 'temp_min': 292.77,
 'temp_max': 292.77,
 'pressure': 1022,
 'humidity': 71,
 'sea_level': 1022,
 'grnd_level': 1001,
 'co': 165.54,
 'no': 0.27,
 'no2': 6.4,
 'o3': 19.11,
 'so2': 0.65,
 'pm2_5': 7.87,
 'pm10': 7.91,
 'nh3': 0.27,
 'city name': 'Chicago',
 'aqi': 1}

Convert the merged dictionary into a dataframe.

In [13]:
pd.DataFrame(weather_chicago, index=[city_name])

,lon,lat,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level,co,no,no2,o3,so2,pm2_5,pm10,nh3,city name,aqi
Chicago,-87.65,41.85,292.77,292.64,292.77,292.77,1022,71,1022,1001,165.54,0.27,6.4,19.11,0.65,7.87,7.91,0.27,Chicago,1


Create a list of different cities I am interested in exploring.

In [14]:
cities = [
    'Chicago,IL,US',
    'Boston,MA,US',
    'Austin,TX,US',
    'New York City,NY,US',
    'Atlanta,GA,US',
    'Los Angeles,CA,US',
    'Seattle,WA,US',
    'Minneapolis,MN,US',
    'Denver,CO,US',
    'Washington,DC,US'
]

Retrieve API for weather and air quality data from all cities in the list and storing them as lists of dictionaries.

In [15]:
all_city_name = []
all_weather_coord = []
all_weather_main = []
all_air_quality_aqi = []
all_air_quality_components = []
for index in range(len(cities)):
    all_city_name.append(cities[index].split(',')[0])
    all_weather = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={cities[index].split(',')[0]},{cities[index].split(',')[2]}&appid={my_key}')
    all_weather_coord.append(all_weather.json()['coord'])
    all_weather_main.append(all_weather.json()['main'])
    all_air_quality = requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution?lat={all_weather_coord[index]['lat']}&lon={all_weather_coord[index]['lon']}&appid={my_key}')
    all_air_quality_aqi.append(all_air_quality.json()['list'][0]['main']['aqi'])
    all_air_quality_components.append(all_air_quality.json()['list'][0]['components'])

Merge lists of dictionaries into a single dataframe.

In [16]:
all_weather_data = {'City' : all_city_name, 'Weather Coordinates' : all_weather_coord, 'Weather Main' : all_weather_main, 'Air Quality Components' : all_air_quality_components, 
                    'Air Quality API' : all_air_quality_aqi}
pd.DataFrame(all_weather_data)

,City,Weather Coordinates,Weather Main,Air Quality Components,Air Quality API
0,Chicago,"{'lon': -87.65, 'lat': 41.85}","{'temp': 292.77, 'feels_like': 292.64, 'temp_m...","{'co': 165.54, 'no': 0.27, 'no2': 6.4, 'o3': 1...",1
1,Boston,"{'lon': -71.0598, 'lat': 42.3584}","{'temp': 297.01, 'feels_like': 297.33, 'temp_m...","{'co': 190.97, 'no': 0.71, 'no2': 7.05, 'o3': ...",1
2,Austin,"{'lon': -97.7431, 'lat': 30.2672}","{'temp': 300.85, 'feels_like': 302.47, 'temp_m...","{'co': 149.41, 'no': 3.09, 'no2': 8.42, 'o3': ...",1
3,New York City,"{'lon': -74.006, 'lat': 40.7143}","{'temp': 300.54, 'feels_like': 302.58, 'temp_m...","{'co': 185.48, 'no': 0.99, 'no2': 4.8, 'o3': 7...",2
4,Atlanta,"{'lon': -84.388, 'lat': 33.749}","{'temp': 298.13, 'feels_like': 298.69, 'temp_m...","{'co': 126.4, 'no': 0.41, 'no2': 3.27, 'o3': 2...",1
5,Los Angeles,"{'lon': -118.2437, 'lat': 34.0522}","{'temp': 291.63, 'feels_like': 291.57, 'temp_m...","{'co': 82.07, 'no': 0, 'no2': 0.71, 'o3': 57.1...",1
6,Seattle,"{'lon': -122.3321, 'lat': 47.6062}","{'temp': 289.46, 'feels_like': 289.29, 'temp_m...","{'co': 79.84, 'no': 0.01, 'no2': 2.4, 'o3': 28...",1
7,Minneapolis,"{'lon': -93.2638, 'lat': 44.98}","{'temp': 291.56, 'feels_like': 291.02, 'temp_m...","{'co': 230.34, 'no': 0.85, 'no2': 2.46, 'o3': ...",2
8,Denver,"{'lon': -104.9847, 'lat': 39.7392}","{'temp': 290.83, 'feels_like': 290.38, 'temp_m...","{'co': 148.01, 'no': 0.24, 'no2': 3.24, 'o3': ...",2
9,Washington,"{'lon': -120.5015, 'lat': 47.5001}","{'temp': 289.73, 'feels_like': 288.91, 'temp_m...","{'co': 124.57, 'no': 0.02, 'no2': 2.21, 'o3': ...",1


Next, I will use the 5-day forecast endpoint to retrieve forecasts for Chicago (https://openweathermap.org/forecast5).

In [17]:
# Retrieve the API for 5-day weather forecast for Chicago
five_day_weather = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={weather_coord['lat']}&lon={weather_coord['lon']}&appid={my_key}')
five_day_weather.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1753974000,
   'main': {'temp': 293.15,
    'feels_like': 292.85,
    'temp_min': 293.15,
    'temp_max': 293.15,
    'pressure': 1022,
    'sea_level': 1022,
    'grnd_level': 1001,
    'humidity': 63,
    'temp_kf': 0},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': {'all': 66},
   'wind': {'speed': 6.45, 'deg': 43, 'gust': 8.7},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2025-07-31 15:00:00'},
  {'dt': 1753984800,
   'main': {'temp': 293.24,
    'feels_like': 292.82,
    'temp_min': 293.24,
    'temp_max': 293.42,
    'pressure': 1022,
    'sea_level': 1022,
    'grnd_level': 1001,
    'humidity': 58,
    'temp_kf': -0.18},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': {'all': 58},
   'wind': {'speed': 6.38, 'deg': 27, 'gust': 7.47},
   'visibil

Retrieve date and temperature data from the 5 day weather forecast for Chicago and convert it into a dataframe.

In [18]:
list_of_weather_data = five_day_weather.json()['list']
five_day_temp = []
five_day_date = []
for index in range(len(list_of_weather_data)):
    five_day_temp.append(list_of_weather_data[index]['main']['temp'])
    five_day_date.append(pd.to_datetime(list_of_weather_data[index]['dt_txt']))
    five_day_temp[index] = round(five_day_temp[index] - 273.15, 2)

five_day_date_temp = {'Date and Time' : five_day_date, 'Temperature (Celsius)' : five_day_temp}
five_day_date_temp_df = pd.DataFrame(five_day_date_temp)
five_day_date_temp_df

,Date and Time,Temperature (Celsius)
0,2025-07-31 15:00:00,20.00
1,2025-07-31 18:00:00,20.09
2,2025-07-31 21:00:00,23.11
3,2025-08-01 00:00:00,22.31
4,2025-08-01 03:00:00,18.60
5,2025-08-01 06:00:00,16.24
6,2025-08-01 09:00:00,19.43
7,2025-08-01 12:00:00,19.58
8,2025-08-01 15:00:00,22.09
9,2025-08-01 18:00:00,23.83


Plot a line chart of temperature against date.

In [19]:
fig6 = px.line(x = five_day_date, y = five_day_temp, markers=True, width = 800, height = 500)
fig6.update_layout(title = {'text' : 'Chicago 5-Day Temperature Forecast', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title_text = 'Date', yaxis_title_text = 'Temperature (°C)')
fig6.update_traces(hovertemplate = 'Date/Time: %{x}<br>Temperature: %{y:.1f} °C<extra></extra>', marker=dict(color='red'))
fig6.show()

Export dataframe as csv file and plot as png file.

In [20]:
# five_day_date_temp_df.to_csv('Chicago 5-day Temperature Forecast.csv', index=False)
# fig6.write_image("Chicago 5-Day Temperature Forecast.png", width = 800, height = 500, scale = 3)

### Part B

In this section, I will request and analyze air quality data from OpenAQ's API (https://openaq.org/).

In [21]:
# The API key is stored separately on a config file for security reasons
with open("config.json") as f:
    config = json.load(f)
    openaq_api = config["openaq_api"]

In [22]:
openaq_url = 'https://api.openaq.org/v3/locations'
open_aq_response = requests.get(openaq_url, headers = {'X-API-Key' : openaq_api})
open_aq_response.json()

{'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 100,
  'found': '>100'},
 'results': [{'id': 3,
   'name': 'NMA - Nima',
   'locality': None,
   'timezone': 'Africa/Accra',
   'country': {'id': 152, 'code': 'GH', 'name': 'Ghana'},
   'owner': {'id': 4, 'name': 'Unknown Governmental Organization'},
   'provider': {'id': 209, 'name': 'Dr. Raphael E. Arku and Colleagues'},
   'isMobile': False,
   'isMonitor': True,
   'instruments': [{'id': 2, 'name': 'Government Monitor'}],
   'sensors': [{'id': 6,
     'name': 'pm10 µg/m³',
     'parameter': {'id': 1,
      'name': 'pm10',
      'units': 'µg/m³',
      'displayName': 'PM10'}},
    {'id': 5,
     'name': 'pm25 µg/m³',
     'parameter': {'id': 2,
      'name': 'pm25',
      'units': 'µg/m³',
      'displayName': 'PM2.5'}}],
   'coordinates': {'latitude': 5.58389, 'longitude': -0.19968},
   'licenses': None,
   'bounds': [-0.19968, 5.58389, -0.19968, 5.58389],
   'distance': None,
   'datetimeFirst': None,
   'da

Research Question: Which countries in Asia have the highest pollution index in terms of PM2.5 and PM10 levels?

To answer this question, I will retrieve API only for asian countries, location name, ID, and sensor data and store them as 4 lists.

In [23]:
open_aq_location_data = open_aq_response.json()['results']
open_aq_country_name = []
open_aq_sensor_data = []
open_aq_location_id = []
open_aq_location_name = []
for open_aq_location_data_value in open_aq_location_data:
    if 'Asia' in open_aq_location_data_value['timezone']:
        open_aq_country_name.append(open_aq_location_data_value['country']['name'])
        open_aq_sensor_data.append(open_aq_location_data_value['sensors'])
        open_aq_location_id.append(open_aq_location_data_value['id'])
        open_aq_location_name.append(open_aq_location_data_value['name'])

print(open_aq_country_name)
print(open_aq_sensor_data)
print(open_aq_location_id)
print(open_aq_location_name)

['India', 'India', 'India', 'India', 'India', 'Vietnam', 'Mongolia', 'China', 'Bangladesh', 'Mongolia', 'Singapore', 'Mongolia', 'China', 'Israel', 'Mongolia', 'Mongolia', 'Thailand', 'India', 'China', 'Thailand', 'Mongolia', 'Thailand', 'Mongolia', 'Thailand', 'India', 'Thailand', 'Thailand', 'China', 'China', 'China']
[[{'id': 23, 'name': 'pm25 µg/m³', 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}}], [{'id': 13866, 'name': 'no2 µg/m³', 'parameter': {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'displayName': 'NO₂ mass'}}, {'id': 24, 'name': 'o3 µg/m³', 'parameter': {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': 'O₃ mass'}}, {'id': 13864, 'name': 'pm25 µg/m³', 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}}], [{'id': 27, 'name': 'co µg/m³', 'parameter': {'id': 4, 'name': 'co', 'units': 'µg/m³', 'displayName': 'CO mass'}}, {'id': 28, 'name': 'no2 µg/m³', 'parameter': {'id': 5, 'name': 'no2', 'units': 'µg/m³', '

Filter out data for PM2.5 and PM10 sensor ids, location names, ids, country names, and store them in separate lists.

In [24]:
PM25_country_name = []
PM10_country_name = []
PM25_location_id = []
PM10_location_id = []
PM25_location_name = []
PM10_location_name = []
PM25_sensor_id = []
PM10_sensor_id = []
for index in range(len(open_aq_location_id)):
    for open_aq_sensor_data_value in open_aq_sensor_data[index]:
        if 'pm25' in open_aq_sensor_data_value['name']:
            PM25_sensor_id.append(open_aq_sensor_data_value['id'])
            PM25_location_id.append(open_aq_location_id[index])
            PM25_location_name.append(open_aq_location_name[index])
            PM25_country_name.append(open_aq_country_name[index])
        elif 'pm10' in open_aq_sensor_data_value['name']:
            PM10_sensor_id.append(open_aq_sensor_data_value['id'])
            PM10_location_id.append(open_aq_location_id[index])
            PM10_location_name.append(open_aq_location_name[index])
            PM10_country_name.append(open_aq_country_name[index])

print(PM25_country_name)
print(PM10_country_name)
print(PM25_location_name)
print(PM10_location_name)
print(PM25_location_id)
print(PM10_location_id)
print(PM25_sensor_id)
print(PM10_sensor_id)

['India', 'India', 'India', 'India', 'India', 'India', 'Vietnam', 'Mongolia', 'China', 'Bangladesh', 'Mongolia', 'Singapore', 'China', 'Israel', 'Mongolia', 'Mongolia', 'India', 'India', 'China', 'Mongolia', 'India', 'Thailand', 'China', 'China', 'China']
['India', 'India', 'India', 'India', 'Mongolia', 'Mongolia', 'Mongolia', 'Mongolia', 'Mongolia', 'Thailand', 'India', 'India', 'Thailand', 'Mongolia', 'Thailand', 'Mongolia', 'Thailand', 'India', 'Thailand', 'Thailand']
['SPARTAN - IIT Kanpur', 'Delhi Technological University, Delhi - CPCB', 'IGI Airport', 'Civil Lines', 'R K Puram, Delhi - DPCC', 'R K Puram, Delhi - DPCC', 'SPARTAN - Vietnam Acad. Sci.', 'MNB', 'Beijing US Embassy', 'SPARTAN - Dhaka University', 'Amgalan', 'SPARTAN - NUS', 'SPARTAN - Tsinghua University', 'SPARTAN - Weizmann Institute', 'Bukhiin urguu', 'Baruun 4 zam', 'Punjabi Bagh, Delhi - DPCC', 'Punjabi Bagh, Delhi - DPCC', 'Shenyang', 'Nisekh', 'Income Tax Office, Delhi - CPCB', 'Prabadang Rehabiltation Center',

Retrieve API only for PM2.5 sensor data aggregated by year and store them into separate lists.

In [25]:
PM25_data = []
PM25_date = []
PM25_avg = []
PM25_max = []
PM25_country_name2 = []
PM25_location_id2 = []
PM25_location_name2 = []
PM25_sensor_id2 = []
for index in range(len(PM25_sensor_id)):
        openaq_url2 = (f'https://api.openaq.org/v3/sensors/{PM25_sensor_id[index]}/years')
        PM25_data.append(requests.get(openaq_url2, headers = {'X-API-Key' : openaq_api}).json())
        for PM25_data_value in PM25_data[index]['results']:
                PM25_date.append(PM25_data_value['period']['datetimeFrom']['local'])
                PM25_avg.append(PM25_data_value['value'])
                PM25_max.append(PM25_data_value['summary']['max'])
                PM25_country_name2.append(PM25_country_name[index])
                PM25_location_name2.append(PM25_location_name[index])
                PM25_location_id2.append(PM25_location_id[index])
                PM25_sensor_id2.append(PM25_sensor_id[index])

Combine PM2.5 lists into a single dataframe.

In [26]:
PM25_date_dt = pd.to_datetime(PM25_date, utc = True)
PM25_year = PM25_date_dt.year + 1
asia_yearly_pm25_levels = {'Country' : PM25_country_name2, 'Location Name' : PM25_location_name2, 'Location ID' : PM25_location_id2, 'PM2.5 Sensor ID' : PM25_sensor_id2, 'Year' : PM25_year, 
                           'Annual Mean PM2.5 Levels (μg/m³)' : PM25_avg, 'Annual Max PM2.5 Levels (μg/m³)' : PM25_max}
asia_yearly_pm25_levels_df = pd.DataFrame(asia_yearly_pm25_levels).sort_values(by = ['Country', 'Year'])
asia_yearly_pm25_levels_df

,Country,Location Name,Location ID,PM2.5 Sensor ID,Year,Annual Mean PM2.5 Levels (μg/m³),Annual Max PM2.5 Levels (μg/m³)
10,China,Beijing US Embassy,21,40,2016,73.2,782.0
45,China,Chengdu,142,218,2016,71.1,281.0
51,China,Shanghai,143,219,2016,42.5,209.0
59,China,Guangzhou,144,220,2016,32.4,266.0
11,China,Beijing US Embassy,21,40,2017,61.8,684.0
...,...,...,...,...,...,...,...
29,Mongolia,Baruun 4 zam,48,69,2019,162.0,4305.0
38,Mongolia,Nisekh,59,1507,2019,143.0,513.5
42,Thailand,Prabadang Rehabiltation Center,135,5077838,2022,20.0,20.0
43,Thailand,Prabadang Rehabiltation Center,135,5077838,2023,23.0,110.0


Plot a line chart of annual mean pm2.5 levels against year for the different asian countries.

In [27]:
asia_yearly_pm25_levels_df['Location and Country'] = asia_yearly_pm25_levels_df['Location Name'] + ', ' + asia_yearly_pm25_levels_df['Country']
color_map = {'Delhi Technological University, Delhi - CPCB, India': "#003052", 'R K Puram, Delhi - DPCC, India' : "#004d85", 'Income Tax Office, Delhi - CPCB, India' : "#a3bbcc", 
             'Punjabi Bagh, Delhi - DPCC, India' : '#1f77b4', 'MNB, Mongolia': "#380400", 'Amgalan, Mongolia': "#7C0703", 'Bukhiin urguu, Mongolia': "#d30000", 
             'Baruun 4 zam, Mongolia' : "#d86262", 'Nisekh, Mongolia' : "#caa2a2", 'Beijing US Embassy, China': "#073601", 'Chengdu, China': "#058d01", 'Shanghai, China': "#1fc21f", 
             'Guangzhou, China': "#65b451", 'Shenyang, China': "#95b890", 'Prabadang Rehabiltation Center, Thailand': "#A200FF"}
fig7 = px.line(asia_yearly_pm25_levels_df, x = 'Year', y = 'Annual Mean PM2.5 Levels (μg/m³)', color = 'Location and Country', markers = True, custom_data = 'Location and Country', 
               color_discrete_map = color_map, width = 1200, height = 500)
fig7.update_layout(title = {'text' : 'Annual Mean PM2.5 Levels for Asia from 2016 to 2024', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title_text = 'Year', 
                   yaxis_title_text = 'Annual Mean PM2.5 Levels (μg/m³)', legend_title_text = 'Location')
fig7.update_traces(hovertemplate = 'Year: %{x}<br>Annual Mean PM2.5 Levels (μg/m³): %{y:.1f}<br>Location: %{customdata[0]}<extra></extra>')
fig7.show()

The country with the highest mean PM2.5 levels from 2016 to 2018 across all locations is India, going as high as 304 μg/m³ in 2016. <br>
The country with the lowest mean PM2.5 levels from 2016 to 2023 across all locations is China, going as low as 1.6 μg/m³ in 2023. <br>
Overall, there is alot of missing data as not enough locations and countries are reporting data for PM2.5 levels. As a result, analysis is not sufficiently comprehensive for the Asia region.

Plot a bar chart of annual max pm2.5 levels against year for the different asian countries.

In [28]:
asia_yearly_pm25_levels_df['Location and Country'] = asia_yearly_pm25_levels_df['Location Name'] + ', ' + asia_yearly_pm25_levels_df['Country']
color_map = {'Delhi Technological University, Delhi - CPCB, India': "#003052", 'R K Puram, Delhi - DPCC, India' : "#004d85", 'Income Tax Office, Delhi - CPCB, India' : "#a3bbcc", 
             'Punjabi Bagh, Delhi - DPCC, India' : '#1f77b4', 'MNB, Mongolia': "#380400", 'Amgalan, Mongolia': "#7C0703", 'Bukhiin urguu, Mongolia': "#d30000", 
             'Baruun 4 zam, Mongolia' : "#d86262", 'Nisekh, Mongolia' : "#caa2a2", 'Beijing US Embassy, China': "#073601", 'Chengdu, China': "#058d01", 'Shanghai, China': "#1fc21f", 
             'Guangzhou, China': "#65b451", 'Shenyang, China': "#95b890", 'Prabadang Rehabiltation Center, Thailand': "#A200FF"}
fig8 = px.bar(asia_yearly_pm25_levels_df, x = 'Year', y = 'Annual Max PM2.5 Levels (μg/m³)', color = 'Location and Country', barmode = 'group', custom_data = 'Location and Country', 
              color_discrete_map = color_map, width = 1200, height = 500)
fig8.update_layout(title = {'text' : 'Annual Max PM2.5 Levels for Asia from 2016 to 2024', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title_text = 'Year', 
                   yaxis_title_text = 'Annual Max PM2.5 Levels (μg/m³)', bargap = 0, legend_title_text = 'Location')
fig8.update_traces(hovertemplate = 'Year: %{x}<br>Annual Max PM2.5 Levels (μg/m³): %{y:.0f}<br>Location: %{customdata[0]}<extra></extra>')
fig8.show()

The location with the highest max PM2.5 levels is Beijing, China in 2018, reaching a level of 5032 μg/m³. <br>
Baruun 4 zam, Mongolia has reported very high max PM2.5 levels from 2017 to 2019, PM2.5 levels of 3804, 3631, and 4305 μg/m³, respectively.

Retrieve API only for PM10 sensor data aggregated by year and store them into separate lists.

In [29]:
PM10_data = []
PM10_date = []
PM10_avg = []
PM10_max = []
PM10_country_name2 = []
PM10_location_id2 = []
PM10_location_name2 = []
PM10_sensor_id2 = []
for index in range(len(PM10_sensor_id)):
        openaq_url2 = (f'https://api.openaq.org/v3/sensors/{PM10_sensor_id[index]}/years')
        PM10_data.append(requests.get(openaq_url2, headers = {'X-API-Key' : openaq_api}).json())
        for PM10_data_value in PM10_data[index]['results']:
                PM10_date.append(PM10_data_value['period']['datetimeFrom']['local'])
                PM10_avg.append(PM10_data_value['value'])
                PM10_max.append(PM10_data_value['summary']['max'])
                PM10_country_name2.append(PM10_country_name[index])
                PM10_location_name2.append(PM10_location_name[index])
                PM10_location_id2.append(PM10_location_id[index])
                PM10_sensor_id2.append(PM10_sensor_id[index])

Combine PM10 lists into a single dataframe.

In [30]:
PM10_date_dt = pd.to_datetime(PM10_date, utc = True)
PM10_year = PM10_date_dt.year + 1
asia_yearly_pm10_levels = {'Country' : PM10_country_name2, 'Location Name' : PM10_location_name2, 'Location ID' : PM10_location_id2, 'PM10 Sensor ID' : PM10_sensor_id2, 'Year' : PM10_year, 
                           'Annual Mean PM10 Levels (μg/m³)' : PM10_avg, 'Annual Max PM10 Levels (μg/m³)' : PM10_max}
asia_yearly_pm10_levels_df = pd.DataFrame(asia_yearly_pm10_levels).sort_values(by = ['Country', 'Year'])
asia_yearly_pm10_levels_df

,Country,Location Name,Location ID,PM10 Sensor ID,Year,Annual Mean PM10 Levels (μg/m³),Annual Max PM10 Levels (μg/m³)
0,India,"R K Puram, Delhi - DPCC",17,399,2016,250.0,973.000000
23,India,"Punjabi Bagh, Delhi - DPCC",50,395,2016,248.0,999.000000
39,India,"Income Tax Office, Delhi - CPCB",103,13862,2016,289.0,975.000000
1,India,"R K Puram, Delhi - DPCC",17,399,2017,259.0,996.000000
24,India,"Punjabi Bagh, Delhi - DPCC",50,395,2017,242.0,989.000000
40,India,"Income Tax Office, Delhi - CPCB",103,13862,2017,196.0,966.000000
2,India,"R K Puram, Delhi - DPCC",17,399,2018,328.0,893.000000
25,India,"Punjabi Bagh, Delhi - DPCC",50,395,2018,334.0,887.000000
41,India,"Income Tax Office, Delhi - CPCB",103,13862,2018,254.0,590.000000
3,Mongolia,MNB,19,5138,2016,150.0,1309.000000


Plot a line chart of annual mean pm10 levels against year for the different asian countries.

In [31]:
asia_yearly_pm10_levels_df['Location and Country'] = asia_yearly_pm10_levels_df['Location Name'] + ', ' + asia_yearly_pm10_levels_df['Country']
color_map = {'R K Puram, Delhi - DPCC, India' : "#004d85", 'Income Tax Office, Delhi - CPCB, India' : "#a3bbcc", 'Punjabi Bagh, Delhi - DPCC, India' : '#1f77b4', 'MNB, Mongolia': "#380400", 
             'Amgalan, Mongolia': "#7C0703", '100 ail, Mongolia': "#a70000", 'Bukhiin urguu, Mongolia': "#d30000", 'Baruun 4 zam, Mongolia' : "#d86262", 'Nisekh, Mongolia' : "#caa2a2", 
             'Misheel expo, Mongolia' : "#cabbbb", 'Lat Phrao Rd., Wang Thonglang, Thailand': "#380144", 'Khlong Nueng, Khlong Luang, Thailand': "#660069", 'Din Daeng, Din Daeng, '
             'Thailand': "#9300ce", 'Prabadang Rehabiltation Center, Thailand': "#BE8CDB"}
fig9 = px.line(asia_yearly_pm10_levels_df, x = 'Year', y = 'Annual Mean PM10 Levels (μg/m³)', color = 'Location and Country', markers = True, custom_data = 'Location and Country', 
               color_discrete_map = color_map, width = 1200, height = 500)
fig9.update_layout(title = {'text' : 'Annual Mean PM10 Levels for Asia from 2016 to 2024', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title_text = 'Year', 
                   yaxis_title_text = 'Annual Mean PM10 Levels (μg/m³)', legend_title_text = 'Location')
fig9.update_traces(hovertemplate = 'Year: %{x}<br>Annual Mean PM10 Levels (μg/m³): %{y:.1f}<br>Location: %{customdata[0]}<extra></extra>')
fig9.show()

The country with the highest mean PM10 levels from 2016 to 2018 across all locations is India, going as high as 334 μg/m³ in 2018. <br>
The country with the lowest mean PM10 levels from 2016 to 2020 across all locations is Thailand, going as low as 41.6 μg/m³ in 2020. <br>
Overall, there is alot of missing data as not enough locations and countries are reporting data for PM10 levels. As a result, analysis is not sufficiently comprehensive for the Asia region.

Plot a bar chart of annual max pm10 levels against year for the different asian countries.

In [32]:
asia_yearly_pm10_levels_df['Location and Country'] = asia_yearly_pm10_levels_df['Location Name'] + ', ' + asia_yearly_pm10_levels_df['Country']
color_map = {'R K Puram, Delhi - DPCC, India' : "#004d85", 'Income Tax Office, Delhi - CPCB, India' : "#a3bbcc", 'Punjabi Bagh, Delhi - DPCC, India' : '#1f77b4', 'MNB, Mongolia': "#380400", 
             'Amgalan, Mongolia': "#7C0703", '100 ail, Mongolia': "#a70000", 'Bukhiin urguu, Mongolia': "#d30000", 'Baruun 4 zam, Mongolia' : "#d86262", 'Nisekh, Mongolia' : "#caa2a2", 
             'Misheel expo, Mongolia' : "#cabbbb", 'Lat Phrao Rd., Wang Thonglang, Thailand': "#380144", 'Khlong Nueng, Khlong Luang, Thailand': "#660069", 'Din Daeng, Din Daeng, '
             'Thailand': "#9300ce", 'Prabadang Rehabiltation Center, Thailand': "#BE8CDB"}
fig10 = px.bar(asia_yearly_pm10_levels_df, x = 'Year', y = 'Annual Max PM10 Levels (μg/m³)', color = 'Location and Country', barmode = 'group', custom_data = 'Location and Country', 
               color_discrete_map = color_map, width = 1200, height = 500)
fig10.update_layout(title = {'text' : 'Annual Max PM10 Levels for Asia from 2016 to 2024', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title_text = 'Year', 
                    yaxis_title_text = 'Annual Max PM10 Levels (μg/m³)', bargap = 0, legend_title_text = 'Location')
fig10.update_traces(hovertemplate = 'Year: %{x}<br>Annual Max PM10 Levels (μg/m³): %{y:.0f}<br>Location: %{customdata[0]}<extra></extra>')
fig10.show()

The location with the highest max PM10 levels is Bukhiin urguu, Mongolia in 2017, reaching a level of 7209 μg/m³. <br>
As a country, Mongolia has consistently reported very high max PM10 levels from 2016 to 2019, with levels going as high as 5224, 7209, 4188, and 2219 μg/m³, respectively.

Export dataframe as csv files and plots as png files.

In [33]:
# asia_yearly_pm25_levels_df.to_csv('Annual Mean and Max PM2.5 Levels for Asia from 2016 to 2024.csv', index=False)
# asia_yearly_pm10_levels_df.to_csv('Annual Mean and Max PM10 Levels for Asia from 2016 to 2024.csv', index=False)
# fig7.write_image('Annual Mean PM2.5 Levels for Asia from 2016 to 2024.png', width = 1200, height = 500, scale = 3)
# fig8.write_image('Annual Max PM2.5 Levels for Asia from 2016 to 2024.png', width = 1200, height = 500, scale = 3)
# fig9.write_image('Annual Mean PM10 Levels for Asia from 2016 to 2024.png', width = 1200, height = 500, scale = 3)
# fig10.write_image('Annual Max PM10 Levels for Asia from 2016 to 2024.png', width = 1200, height = 500, scale = 3)